In [3]:
pip install azure-eventhub 


StatementMeta(, db0d4172-0630-4c70-9c98-683bce49876d, 5, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests, json, time
from statistics import mean
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime, timezone, timedelta
import notebookutils
import sys
import uuid

# ==== FABRIC EVENTSTREAM CONNECTION DETAILS ====
CONNECTION_STRING ="Endpoint=sb://esehbl6c19ci24lun7kr7q.servicebus.windows.net/;SharedAccessKeyName=key_28847a70-eb28-4023-b148-535b0a72f4bd;SharedAccessKey=xju2lupaEBR0vYo55HAcKhxklPPiPX3VZ+AEhJq9UHg=;EntityPath=es_648849a4-b428-4a34-ba8c-1a1be10efe93"
producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING)
IST = timezone(timedelta(hours=5, minutes=30))
# ==== TOMTOM TRAFFIC API CONFIG ====
API_KEY = "NhWEwanvL7p8Bd6hGqVc3lhVWVOcFp1P"
points = {
    "Connaught Place": (28.6139, 77.2090),
    "Noida": (28.5355, 77.3910),
    "West Delhi": (28.7041, 77.1025),
    "Gurugram": (28.4595, 77.0266),
    "New Delhi":(28.63,77.22)
}

# Get the current datetime object
now = datetime.now()
# Convert to Unix timestamp (float, seconds since epoch)
timestamp_seconds = now.timestamp()
dt_str = datetime.now(IST).isoformat()
dt_obj = datetime.fromisoformat(dt_str)
formatted_dt = dt_obj.strftime("%Y-%m-%d %H:%M")
# Convert to milliseconds and cast to an integer
timestamp_millis = int(timestamp_seconds * 1000)
print(timestamp_millis)
#run_id = notebookutils.getParameter("runId")
#print(f"Pipeline Run ID: {run_id}")
# ==== FETCH TRAFFIC DATA ====
events = []
for city, (lat, lon) in points.items():
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/relative0/10/json?point={lat},{lon}&key={API_KEY}"
    r = requests.get(url)
    data = r.json()["flowSegmentData"]

    current = data["currentSpeed"]
    free = data["freeFlowSpeed"]
    t_cur = data["currentTravelTime"]
    t_free = data["freeFlowTravelTime"]

    congestion_index = (1 - current / free) * 100
    delay = (t_cur - t_free) / 60
    print("data",data)
    event = {
        "flowSegmentData": data,
        "city": city,
        "lat": lat,
        "long": lon,
       # "AverageSpeed_kmph": current,
       # "CongestionIndex_percent": round(congestion_index, 2),
       # "PeakHourDelay_mins": round(delay, 2),
       # "dateTime": formatted_dt,        
       # "timestamp_millis": timestamp_millis
    }
    events.append(event)
    
# ==== SEND TO FABRIC EVENTSTREAM ====
event_batch = producer.create_batch()
for e in events:
    event_batch.add(EventData(json.dumps(e)))

producer.send_batch(event_batch)
producer.close()
print(events)

print("✅ Sent traffic data to Fabric Eventstream successfully!")



StatementMeta(, db0d4172-0630-4c70-9c98-683bce49876d, 6, Finished, Available, Finished)

1769505964085
data {'frc': 'FRC4', 'currentSpeed': 15, 'freeFlowSpeed': 24, 'currentTravelTime': 429, 'freeFlowTravelTime': 268, 'confidence': 0.76889, 'roadClosure': True, 'coordinates': {'coordinate': [{'latitude': 28.60941906113328, 'longitude': 77.20872623315284}, {'latitude': 28.60976770669141, 'longitude': 77.20874098530243}, {'latitude': 28.6100520381346, 'longitude': 77.20875573745201}, {'latitude': 28.610245197643412, 'longitude': 77.20876646628807}, {'latitude': 28.611056537285588, 'longitude': 77.20882011046837}, {'latitude': 28.61121614072751, 'longitude': 77.20882815709541}, {'latitude': 28.611501792755323, 'longitude': 77.20884559145401}, {'latitude': 28.612027472870036, 'longitude': 77.2088791190667}, {'latitude': 28.61206507390843, 'longitude': 77.2088804601712}, {'latitude': 28.612090533703714, 'longitude': 77.20878255954216}, {'latitude': 28.612122689563485, 'longitude': 77.20873159757087}, {'latitude': 28.61214822292795, 'longitude': 77.20871282210777}, {'latitude': 